In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from pvlib import pvsystem
from pvlib.ivtools.sdm import fit_cec_sam
from pvlib.ivtools.sdm import fit_pvsyst_sandia
from pvlib.pvsystem import calcparams_desoto
from pvlib.pvsystem import calcparams_cec


def module_cec_parameters_from_datasheet(datasheet):
    celltype_translation = {                                                        
        "Mono": "monoSi",                                                           
        "Poly": "polySi",                                                           
    } 
    fit_parameters = {                                                              
        "celltype": celltype_translation[datasheet["cell_type"]],                        
        "v_mp": datasheet["v_mp"],                                                       
        "i_mp": datasheet["i_mp"],                                                       
        "v_oc": datasheet["v_oc"],                                                                                         
        "i_sc": datasheet["i_sc"],                                                       
        "alpha_sc": datasheet["alpha_i_sc_percent"] / 100 * datasheet["i_sc"],                
        "beta_voc": datasheet["beta_v_oc_percent"] / 100 * datasheet["v_oc"],                 
        "gamma_pmp": datasheet["gamma_p_mp_percent"],                                    
        "cells_in_series": datasheet["cells_in_series"],                                 
        "temp_ref": datasheet["t_ref"],                                                  
    }
    parameters = fit_cec_sam(**fit_parameters)                                      
    names = ["I_L_ref", "I_o_ref", "R_s", "R_sh_ref", "a_ref", "Adjust"]            
    parameters = dict(zip(names, parameters))                         
    parameters["alpha_sc"] = fit_parameters["alpha_sc"] * (1.0 - parameters["Adjust"] / 100)
    del parameters["Adjust"]
    parameters["temp_ref"] = datasheet["t_ref"]
    parameters["irrad_ref"] = datasheet["irrad_ref"]
    return parameters

# Datasheet module parameters
notc = {
    'cell_type': 'Mono',
    'cells_in_series': 72,
    'n_cells': 144,
    'rated_power': 540,
    'v_oc': 46.4,
    'v_mp': 38.6,
    'i_sc': 11.21,
    'i_mp': 10.45,
    't_ref': 20.0,
    'irrad_ref': 800,
    'alpha_i_sc_percent': 0.05,
    'beta_v_oc_percent': -0.26,
    'gamma_p_mp_percent': -0.34
}
stc = {
    'cell_type': 'Mono',
    'cells_in_series': 72,
    'n_cells': 144,
    'rated_power': 540,
    'v_oc': 49.2,
    'v_mp': 41.3,
    'i_sc': 13.90,
    'i_mp': 13.08,
    't_ref': 25.0,
    'irrad_ref': 1000,
    'alpha_i_sc_percent': 0.05,
    'beta_v_oc_percent': -0.26,
    'gamma_p_mp_percent': -0.34
}
module_conditions = {
    "stc": module_cec_parameters_from_datasheet(stc),
    "notc": module_cec_parameters_from_datasheet(notc),
}

# Plot I-V curves
plt.figure()
for irradiance, temperature in [(1000, 25), (800, 20)]:
    for name, module_parameters in module_conditions.items():
        sd_parameters = calcparams_desoto(
            effective_irradiance=irradiance,
            temp_cell=temperature,
            **module_parameters,
        )
        names = ['photocurrent', 'saturation_current', 'resistance_series', 'resistance_shunt', 'nNsVth']
        sd_parameters = dict(zip(names, sd_parameters))
        curve_info = pvsystem.singlediode(
            **sd_parameters,
            ivcurve_pnts=100,
            method='lambertw'
        )
        label = f"{irradiance}, {temperature}, {name}"
        plt.plot(curve_info['v'], curve_info['i'], label=label)
        v_mp = curve_info['v_mp']
        i_mp = curve_info['i_mp']
        plt.plot([v_mp], [i_mp], ls='', marker='o', c='k')
        print(label, v_mp * i_mp)

plt.legend(loc=(1.0, 0))
plt.xlabel('Module voltage [V]')
plt.ylabel('Module current [A]')
plt.show()